In [43]:
%load_ext autoreload 
%autoreload 2
%matplotlib inline

import matplotlib as mpl
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.linalg as la
from numbers import Number

import scipy.linalg
import scipy.stats

import mdpy

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [34]:
import toolz

In [2]:
A = np.eye(3)

In [3]:
A[0, None].shape

(1, 3)

In [4]:
np.squeeze(A[0, None])

array([ 1.,  0.,  0.])

In [5]:
B = np.arange(8).reshape(2, 2, 2)

In [6]:
B

array([[[0, 1],
        [2, 3]],

       [[4, 5],
        [6, 7]]])

In [7]:
np.squeeze(B[1, None, 0])

array([4, 5])

In [8]:
B[1, 1, 0]

6

In [10]:
A = np.arange(9).reshape(3, 3)

In [11]:
A.sum(axis=1, keepdims=True)

array([[ 3],
       [12],
       [21]])

In [12]:
(A / A.sum(axis=1, keepdims=True)).sum(axis=1)

array([ 1.,  1.,  1.])

In [13]:
def stochastic(ns, rv=None):
    if rv is None:
        rv = scipy.stats.uniform()
    ret = np.abs(rv.rvs((ns, ns)))
    ret = ret/ret.sum(axis=1, keepdims=True)
    return ret

In [14]:
tmat = stochastic(3)

In [15]:
rmat = np.eye(3)

In [16]:
print(tmat)
print(rmat)

[[ 0.33385834  0.27981766  0.386324  ]
 [ 0.22196796  0.62884377  0.14918827]
 [ 0.26654253  0.37494244  0.35851503]]
[[ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0.  1.]]


In [89]:
class MarkovProcess:
    """A class implementing Markov processes, which are like MDPs where you
    don't make any decisions.
    It requires two arrays, one for the transition probabilities (`T`) and
    another of the same shape for the expected rewards (`R`).

    For example, given state `s` and next state `sp`, the probability of the
    transition `(s, sp)` is `T[s, sp]`, with reward `R[s, sp]`.
    """
    def __init__(self, transitions, rewards):
        T = np.array(transitions)
        R = np.array(rewards)
        # Check that shapes are valid
        assert(2 == T.ndim == R.ndim)
        assert(T.shape == R.shape)
        assert(T.shape[0] == T.shape[1])
        # Check that transition probabilities sum to one
        assert(np.allclose(1, np.einsum('ij->i', T)))

        # Initialize variables
        self.T = T
        self.R = R
        self._states = np.arange(len(T))

    @classmethod
    def from_unnormalized(cls, transitions, rewards=None):
        """Create a Markov Process using an arbitrary transition matrix by
        taking the absolute value and normalizing the transition probabilities.
        """
        pass
    
    def prob(self, s, sp=None):
        return np.squeeze(self.T[s, sp])

    def transition(self, s):
        return np.random.choice(self._states, p=self.T[s])

    def step(self, s):
        sp = np.random.choice(self._states, p=self.T[s])
        r  = self.reward(s, sp)
        return (sp, r)
    
    def reward(self, s, sp):
        r = self.R[s, sp]
        if isinstance(r, Number):
            return r
        elif isinstance(r, scipy.stats._distn_infrastructure.rv_frozen):
            return r.rvs()
        elif isinstance(r, scipy.stats._distn_infrastructure.rv_generic):
            return r.rvs()
        elif callable(r):
            return r(s, sp)
        else:
            raise TypeError("Reward for transition not understood: (%d, %d)"%(s, sp))
    
    def expected_reward(self, s, sp=None):
        """Compute the expected reward either given a state or a transition."""
        def _expectation(rwd):
            """Get the expected value of a reward."""
            if isinstance(rwd, Number):
                return r
            elif isinstance(rwd, scipy.stats._distn_infrastructure.rv_frozen):
                return r.mean()
            elif isinstance(rwd, scipy.stats._distn_infrastructure.rv_generic):
                return r.mean()
            else:
                raise TypeError("Unable to get expected value of reward: %s"%(rwd))
            
        # Compute expectation, either for `(s, sp)` or over possible next states 
        if sp is not None:
            return _expectation(self.R[s, sp])
        else:
            return self.T[s]*[_expectation(r) for r in self.R[s]]

In [64]:
r1

In [61]:
scipy.stats.norm.mean()

0.0

In [47]:
type(r1)

scipy.stats._distn_infrastructure.rv_frozen

In [67]:
P

NameError: name 'P' is not defined

In [69]:
tmat[0]

array([ 0.33385834,  0.27981766,  0.386324  ])

In [72]:
tmat[0] * [i.mean() for i in Q[0]]

array([  2.72450891e-01,   1.85592739e-01,   1.29543337e-04])

In [78]:
[(i, j, x.mean()) for (i, j), x in np.ndenumerate(Q)]

[(0, 0, 0.81606735118416418),
 (0, 1, 0.66326314624079041),
 (0, 2, 0.00033532303249728113),
 (1, 0, 0.058089822241082834),
 (1, 1, 0.68095614560515938),
 (1, 2, 0.18451649845694773),
 (2, 0, 0.69125336736939358),
 (2, 1, 0.99064060701854373),
 (2, 2, 0.13120483043473896)]

In [88]:
print(np.reshape([x.mean()*y for x,y in zip(Q.flat, tmat.flat)], (3,3)))

[[  2.72450891e-01   1.85592739e-01   1.29543337e-04]
 [  1.28940794e-02   4.28215030e-01   2.75276970e-02]
 [  1.84248421e-01   3.71433204e-01   4.70389040e-02]]


In [79]:
tmat

array([[ 0.33385834,  0.27981766,  0.386324  ],
       [ 0.22196796,  0.62884377,  0.14918827],
       [ 0.26654253,  0.37494244,  0.35851503]])

In [73]:
[i.mean() for i in Q]

TypeError: unsupported operand type(s) for +: 'rv_frozen' and 'rv_frozen'

In [45]:
isinstance(r1, (scipy.stats.rv_continuous, scipy.stats.rv_discrete))

False

In [36]:
r1 = scipy.stats.norm()

In [40]:
r1.rvs(1)

array([-0.44041997])

In [70]:
Q = np.reshape([scipy.stats.norm(loc=np.random.random()) for i in range(9)], (3, 3))

In [71]:
Q

array([[<scipy.stats._distn_infrastructure.rv_frozen object at 0x7f0e05dffcc0>,
        <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f0e05ead0f0>]], dtype=object)